# *Python from Scratch LiveLessons*
## Lesson 7: Pie Factory&#8212;Object-Oriented Python
<img src="misc/pie_cover_variation_.png" width="60%"  />

API is short for Application Program Intersface. This is how we make it so one program talks to another. We will conduct the following steps:

1. Read the API documentation for our *Factory Conveyor Belt* we will use (already exists).
1. Create callbacks.
1. Use a Mock Browser (simulation) to handle running our Factory. 
1. Run our Mock Browser.

In Lesson, 8 we will finalize our whole program by hooking up our code to the the actual web browser. By using this Mock Server we will write the callback functions.  By breaking this up into smaller parts we simplify the approach.  

### 7.1 Read the API documentation for our Factory Conveyor Belt we will use (already exists).

In [ ]:
import complete_pie

In [ ]:
help(complete_pie.FactoryConveyorBelt)

Below is our design of basic flow:

<img src="misc/workflow_2.png" width="60%"  />


The callbacks are functions that carry out the task at hand for each operation. The FactoryConveyorBelt holds the data. So we create an instance of FactoryConveryorBelt first:



In [ ]:
 belt = complete_pie.FactoryConveyorBelt()

Next we create a Pie and add it to the belt:

In [ ]:
from complete_pie import ApplePie 
pie = ApplePie(name="Example Apple Pie", recipe_path="misc/ApplePie.txt")
pie.process_recipe()

    
    

In [ ]:
 pie

In [ ]:
belt.fill_pantry(pie, times=10)

In [ ]:
print(belt.get_totals())

In [ ]:
belt.add_pie_order(pie, 3)

In [ ]:
print(belt.get_totals())

### 7.2 Creating callbacks

The belt knows about certain methods:

In [ ]:
 belt.known_callback_methods

We can add a simple callback `echo()` to all of theses. Please note that all callbacks take the `callback_app` and `message` as arguments:

In [ ]:
def echo_callback(callback_app, message):
    callback_app.logger.info("echo {}".format(message))

In [ ]:
for method in belt.known_callback_methods:
    belt.add_callback(method, echo_callback)

### 7.3 Using the mock browser


<img src="misc/workflow_2.png" width="60%"  />

Already created for you is the Mock Browser:

In [ ]:
import random
from pie_logger import get_logger

log = get_logger()


# bake: baketype: 'apple', 'cherry'
# oven:  unique_pie_id: pie.unique_pie_id, heat_time: now()
# restock: ...



class MockApp:
    def __init__(self, belt):
        self.logger = log
        self.belt = belt

def get_random_pie():
    return random.choice(["apple", "cherry", "raseberry"])


def simluate(belt, count):
    
    callback_app = MockApp(belt)
    
    for callback in belt.callbacks.get("restock", []):
        callback(callback_app, {})
        
    for n in range(count):
        callback_app.logger.debug("testing bake callback")
        for callback in belt.callbacks.get("bake", []):
            bake_out = callback(callback_app, {"baketype": get_random_pie()})
            if bake_out:
                for callback in belt.callbacks.get("oven", []):
                    callback_data = {"unique_pie_id": bake_out.get("unique_pie_id"),
                                     "heat_time": 5}
                    callback(callback_app, callback_data)
    callback_app.logger.debug("\n"+belt.get_totals())
    


In [ ]:
help(belt.callbacks['restock'][0])

In [ ]:
help(callable)

### 7.4 Running


In [ ]:
simluate(belt, 4)

In [ ]:
simluate(belt, 2)


### 7.5 Creating more callbacks and Putting it all together

Now we add the callbacks `bake_callback()` and `oven_callback()`:

In [ ]:
from collections import defaultdict
belt.callbacks = defaultdict(list)

In [ ]:
from complete_pie import CherryPie, RaseberryPie, ApplePie


names = ['David', "Sue", "Pap", "Karen", "Julie", "Greg"]

def bake_callback(callback_app, message):
    
    callback_app.logger.info("bake callback")
    baketype = message["baketype"]
    pie_type = None
    if baketype == "apple":
        pie_type = ApplePie
    elif baketype == "cherry":
        pie_type = CherryPie
    elif baketype == "raseberry":
        pie_type = RaseberryPie
    else:
        raise Exception("unknown pie type {}".format(message['baketype']))
    
    # make a pie
    name = "{}'s {} Pie".format(random.choice(names), "Apple")
    pie = pie_type(name, recipe_path="misc/ApplePie.txt")
    pie.process_recipe()
    log.debug("pie data {}".format(pie.__dict__))
    
    # add_pie to the belt
    try:
        callback_app.belt.add_pie(pie)
    except Exception as e:
        return dict(error=str(e))
    
    # get totals
    totals = callback_app.belt.get_totals()
    return {"image_key":pie.image_key,
            "totals":totals,
            "name":pie.name,
            "unique_pie_id": pie.unique_pie_id}

belt.add_callback("bake", bake_callback)


def oven_callback(callback_app, message):
    callback_app.logger.info("message {}".format(message))
    # get pie
    pie = callback_app.belt.pies[message['unique_pie_id']]
    # total time
    total_time = message['heat_time'] + callback_app.belt.oven_heat_time
    # totals
    totals = callback_app.belt.get_totals()
    return {"image_key": pie.image_key,
            "totals": totals,
            "oven_msg": "Oven Heating",
            "name": pie.name,
            "bake_time": pie.bake_time,
            "total_time": total_time,
            "unique_pie_id": pie.unique_pie_id}


belt.add_callback("oven", oven_callback)


In [ ]:
help(ApplePie)

In [ ]:
names = ['David', "Sue", "Pap", "Karen", "Julie", "Greg"]
random.choice(names)

In [ ]:
belt.pies['a9f57164-8ce6-4b6d-acb1-e65f9b5df10e'].__dict__

In [ ]:
simluate(belt, 3)

In [ ]:
simluate(belt, 1)